In [1]:
import torch
import numpy as np

In [2]:
def make_q(r, v, dtype=torch.float64, device='cpu'):
    return torch.tensor(r, dtype=dtype, device=device), torch.tensor(v, dtype=dtype, device=device)

def point_to_q(v):
    return torch.tensor(0, dtype=v.dtype, device=v.device), v

def q_norm(q):
    r, v = q
    return torch.sqrt(r * r + torch.dot(v, v))

def q_normalize(q):
    r, v = q
    norm = torch.sqrt(r * r + torch.dot(v, v))
    return (r/norm, v/norm)

def q_conj(q):
    r, v = q
    return (r, -v)

def q_add(q1, q2):
    r1, v1 = q1
    r2, v2 = q2
    return r1+r2, v1+v2

def q_mul(q1, q2):
    r1, v1 = q1
    r2, v2 = q2
    return r1 * r2 - torch.dot(v1, v2), v1 * r2 + r1 * v2 + torch.cross(v1, v2)

def q_scale(q, a):
    r, v = q
    return (a * r, a * v)

In [3]:
q1 = make_q(np.random.randn(), np.random.randn(3))
q2 = make_q(np.random.randn(), np.random.randn(3))
p = make_q(0, np.random.randn(3))

Check that rotating using a norm-1 quaternion and multiplying by alpha^2 is the same as rotating by norm alpha quaternion. This means that to perform similarity transforms, we don't need to normalize

In [4]:
q_norm(q1), q_norm(p)

(tensor(3.5337, dtype=torch.float64), tensor(1.3584, dtype=torch.float64))

In [5]:
p1 = q_mul(q_mul(q1, p), q_conj(q1))
p1

(tensor(0., dtype=torch.float64),
 tensor([ -0.5199, -16.9135,   1.1793], dtype=torch.float64))

In [6]:
q1n = q_normalize(q1)
p2 = q_mul(q_mul(q1n, p), q_conj(q1n))
tuple(u * q_norm(q1)**2 for u in p2)

(tensor(-1.0398e-15, dtype=torch.float64),
 tensor([ -0.5199, -16.9135,   1.1793], dtype=torch.float64))

Check the Jacobian of quaternion rotation and translation

In [7]:
# Center of rotation
C = make_q(0, np.random.randn(3))[1]

# Offset
b = make_q(0, np.random.randn(3))[1].requires_grad_(True)

# Quaternion
q = tuple(u.clone().detach().requires_grad_(True) for u in q1)

# Starting point
A = make_q(0, np.random.randn(3))[1]

# Transformed point p
A0 = point_to_q(A-C)
B0 = q_mul(q_mul(q, A0), q_conj(q))
B = B0[1] + C + b

In [8]:
# PyTorch derivatives
gamma = make_q(0, [0.33, 0.42, -0.43])[1]
B.backward(gamma)
q[0].grad, q[1].grad, b.grad

(tensor(1.7188, dtype=torch.float64),
 tensor([-1.2083,  0.0327,  5.8198], dtype=torch.float64),
 tensor([ 0.3300,  0.4200, -0.4300], dtype=torch.float64))

In [9]:
# Numerical derivatives. For b it is just gamma
print(f'b: {b.grad} vs {gamma}')

# Compute the derivative for q
G = point_to_q(gamma)
t1 = q_mul(q_mul(G, q), q_conj(A0))
t2 = t1
q_add(t1, t2)

b: tensor([ 0.3300,  0.4200, -0.4300], dtype=torch.float64) vs tensor([ 0.3300,  0.4200, -0.4300], dtype=torch.float64)


(tensor(1.7188, dtype=torch.float64, grad_fn=<AddBackward0>),
 tensor([-1.2083,  0.0327,  5.8198], dtype=torch.float64,
        grad_fn=<AddBackward0>))

In [10]:
allq = [G, q_conj(G), q, q_conj(q), A0, q_conj(A0)]
all_comb = []
for p1 in range(6):
    for p2 in range(6):
        for p3 in range(6):
            all_comb.append( ((p1,p2,p3), q_mul(q_mul(allq[p1], allq[p2]), allq[p3]))) 
            
for i1, (idx1, u1) in enumerate(all_comb):
    for i2, (idx2, u2) in enumerate(all_comb):
        qsum = q_add(u1, u2)
        dr = (qsum[0]-q[0].grad).detach().cpu().numpy() ** 2
        dv = np.sum((qsum[1]-q[1].grad).detach().cpu().numpy() ** 2)
        if dr + dv < 1e-6:
            print(idx1, idx2)

(0, 2, 5) (0, 2, 5)
(0, 2, 5) (1, 2, 4)
(1, 2, 4) (0, 2, 5)
(1, 2, 4) (1, 2, 4)


What is the derivative of quaternion product $q(0,x){\overline q}$ with respect to $x$?

In [11]:
# This is the numerical derivative
X = torch.tensor(np.random.randn(3), requires_grad=True)
fX = q_mul(q_mul(q, point_to_q(X)), q_conj(q))
fX[1].backward(gamma)
print('Num: ', X.grad.detach().cpu().numpy())

# Figure out the analytical derivative
dX = q_mul(q_mul(q_conj(q), point_to_q(gamma)), q)[1]
print('Anl: ', dX.detach().cpu().numpy())

Num:  [-6.28025322  4.89993744 -3.1414461 ]
Anl:  [-6.28025322  4.89993744 -3.1414461 ]


Inverse rotation

In [12]:
def transform_point(q, b, C, A):
    A0 = point_to_q(A-C)
    B0 = q_mul(q_mul(q, A0), q_conj(q))
    return B0[1] + C + b

def transform_point_inv(q, b, C, B):
    B0 = point_to_q(B - C - b)
    A0 = q_scale(q_mul(q_mul(q_conj(q), B0), q), 1.0 / q_norm(q)**4)
    return A0[1] + C

def transform_point_inv_nonorm(q, b, C, B):
    B0 = point_to_q(B - C - b)
    A0 = q_mul(q_mul(q_conj(q), B0), q)
    return A0[1] + C

Now, I would like to backprop through the inverse transform, i.e., if I have some function f(A) of A, where A is transform_point_inv(q, b, C, B), then what is D_B f?

In [13]:
transform_point_inv(q, b, C, transform_point(q, b, C, A)) - A

tensor([1.1102e-16, 1.1102e-16, 3.3307e-16], dtype=torch.float64,
       grad_fn=<SubBackward0>)

In [14]:
transform_point_inv(q, b, C, B)

tensor([-0.6359, -0.4116,  0.8368], dtype=torch.float64,
       grad_fn=<AddBackward0>)

Here are the parameters of the inverse transform

In [19]:
q_inv = q_scale(q_conj(q), 1.0 / q_norm(q)**2)
b_inv = q_mul(q_mul(q_inv, point_to_q(-b)), q_conj(q_inv))[1]
transform_point(q_inv, b_inv, C, B)

tensor([-0.6359, -0.4116,  0.8368], dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [19]:
q[0].grad.zero_()
q[1].grad.zero_()
b.grad.zero_()
fA = transform_point(q, b, C, A)
fA.backward(gamma)
q[0].grad, q[1].grad, b.grad

(tensor(-0.4963, dtype=torch.float64),
 tensor([-3.3621,  2.5635,  8.2509], dtype=torch.float64),
 tensor([ 0.3300,  0.4200, -0.4300], dtype=torch.float64))

In [71]:
q[0].grad.zero_()
q[1].grad.zero_()
b.grad.zero_()
gA = transform_point_inv(q, b, C, A)
gA.backward(gamma)
q[0].grad, q[1].grad, b.grad

(tensor(-0.5484, dtype=torch.float64),
 tensor([ 0.3349, -0.4979,  0.0287], dtype=torch.float64),
 tensor([-0.0880,  0.0848,  0.1131], dtype=torch.float64))

In [95]:
# This is the b gradient, it is correct
grad_finv_b = q_mul(q_mul(q, point_to_q(-gamma)), q_conj(q))[1] / q_norm(q)**4
grad_finv_q = q_add(
    q_scale(q_mul(q_mul(point_to_q(A - C - b), q), q_conj(G)), 2 / q_norm(q)**4),
    q_scale(q, -4 * torch.dot(q_mul(q_mul(q_conj(q), point_to_q(A - C - b)), q)[1], gamma) / q_norm(q)**6))

grad_finv_q, grad_finv_b

((tensor(-0.5484, dtype=torch.float64, grad_fn=<AddBackward0>),
  tensor([ 0.3349, -0.4979,  0.0287], dtype=torch.float64,
         grad_fn=<AddBackward0>)),
 tensor([-0.0880,  0.0848,  0.1131], dtype=torch.float64,
        grad_fn=<DivBackward0>))